In [1]:
import pandas as pd

In [2]:
kospi1 = pd.read_csv("./데이터/코스피200연결.csv", encoding = "euc-kr")
kospi2 = pd.read_csv("./데이터/코스피200개별.csv", encoding = "euc-kr")
st13 = pd.read_csv("./데이터/코스피주식(2013).csv", encoding = "euc-kr")
st14 = pd.read_csv("./데이터/코스피주식(2014).csv", encoding = "euc-kr")
st15 = pd.read_csv("./데이터/코스피주식(2015).csv", encoding = "euc-kr")
st16 = pd.read_csv("./데이터/코스피주식(2016).csv", encoding = "euc-kr")
st17 = pd.read_csv("./데이터/코스피주식(2017).csv", encoding = "euc-kr")
st18 = pd.read_csv("./데이터/코스피주식(2018).csv", encoding = "euc-kr")
st19 = pd.read_csv("./데이터/코스피주식(2019).csv", encoding = "euc-kr")
st20 = pd.read_csv("./데이터/코스피주식(2020).csv", encoding = "euc-kr")
st21 = pd.read_csv("./데이터/코스피주식(2021).csv", encoding = "euc-kr")
st22 = pd.read_csv("./데이터/코스피주식(2022).csv", encoding = "euc-kr")


In [3]:
kospi1.columns = ['회사명', '종목코드', '회계년도', '거래월', '종가', '자산', '부채', '당기순이익', '현금흐름', '매출액', '비유동자산', '매출총이익', '법인세비용', '이자비용', '감가상각비', '유동자산',
       '유동부채', '현금및현금성자산']
kospi2.columns = ['회사명', '종목코드', '회계년도', '거래월', '종가', '자산', '부채', '당기순이익', '현금흐름', '매출액', '비유동자산', '매출총이익', '법인세비용', '이자비용', '감가상각비', '유동자산',
       '유동부채', '현금및현금성자산']

In [4]:
st13['회계년도'] = '2013/12'
st14['회계년도'] = '2014/12'
st15['회계년도'] = '2015/12'
st16['회계년도'] = '2016/12'
st17['회계년도'] = '2017/12'
st18['회계년도'] = '2018/12'
st19['회계년도'] = '2019/12'
st20['회계년도'] = '2020/12'
st21['회계년도'] = '2021/12'
st22['회계년도'] = '2022/12'


In [5]:
df = pd.concat([st13, st14, st15, st16, st17, st18, st19, st20, st21, st22], axis=0)

In [6]:
df['종목코드'] = df['종목코드'].apply(lambda x: x.lstrip('0'))

In [7]:
kospi = kospi1.fillna(kospi2)

In [8]:
kospi['종목코드'] = kospi['종목코드'].astype(str)

In [9]:
kospi = pd.merge(kospi, df[['종목코드', '회계년도', '상장주식수', '시가총액']], on=['종목코드', '회계년도'], how='left')

In [10]:
kospi['pbr'] = (kospi['종가']*kospi['상장주식수'])/(kospi['자산']-kospi['부채'])
kospi['per'] = kospi['종가']/(kospi['당기순이익']/kospi['상장주식수'])
kospi['pcr'] = kospi['종가']/(kospi['현금흐름']/kospi['상장주식수'])
kospi['psr'] = kospi['종가']/(kospi['매출액']/kospi['상장주식수'])
kospi['roe'] = kospi['당기순이익']/(kospi['자산']-kospi['부채'])
kospi['roa'] = kospi['당기순이익']/kospi['자산']
kospi['roic'] = kospi['당기순이익']/(kospi['자산']-kospi['비유동자산'])
kospi['gpa'] = kospi['매출총이익']/kospi['자산']

In [11]:
# 분자(EBIT)
magic_ebit = kospi['당기순이익'] + kospi['법인세비용'] + kospi['이자비용']

# 분모
magic_cap = kospi['시가총액']
magic_debt = kospi['부채']

## 분모: 여유자금
magic_excess_cash = kospi['유동부채'] - kospi['유동자산'] + kospi[
    '현금및현금성자산']
magic_excess_cash[magic_excess_cash < 0] = 0
magic_excess_cash_final = kospi['현금및현금성자산'] - magic_excess_cash

magic_ev = magic_cap + magic_debt - magic_excess_cash_final

# 이익수익률
magic_ey = magic_ebit / magic_ev
kospi['매직'] = magic_ey

In [12]:
kospi = kospi.dropna()

In [13]:
len(kospi['회사명'].unique())

179

In [14]:
kospi

,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
120,(주)DB하이텍,990,2013/12,1.0,6550.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1419.290470,-3499.345087,10239.594604,588.573379,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
121,(주)DB하이텍,990,2013/12,2.0,6890.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1492.963563,-3680.990481,10771.115545,619.125280,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
122,(주)DB하이텍,990,2013/12,3.0,6250.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1354.284799,-3339.069740,9770.605538,561.615820,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
123,(주)DB하이텍,990,2013/12,4.0,5970.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1293.612840,-3189.479415,9332.882410,536.455431,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
124,(주)DB하이텍,990,2013/12,5.0,8490.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1839.660471,-4535.792334,13272.390563,762.898930,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20812,효성티앤씨(주),298020,2022/12,8.0,313500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,895.220804,70724.639905,4624.426176,152.737768,0.012658,0.004441,0.008951,0.100036,0.000020
20813,효성티앤씨(주),298020,2022/12,9.0,259000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,739.592307,58429.606811,3820.498819,126.185270,0.012658,0.004441,0.008951,0.100036,0.000020
20814,효성티앤씨(주),298020,2022/12,10.0,261500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,746.731229,58993.599155,3857.376221,127.403274,0.012658,0.004441,0.008951,0.100036,0.000020
20815,효성티앤씨(주),298020,2022/12,11.0,357000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,1019.438044,80538.106686,5266.092966,173.931047,0.012658,0.004441,0.008951,0.100036,0.000020


In [15]:
kospi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18345 entries, 120 to 20816
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회사명       18345 non-null  object 
 1   종목코드      18345 non-null  object 
 2   회계년도      18345 non-null  object 
 3   거래월       18345 non-null  float64
 4   종가        18345 non-null  float64
 5   자산        18345 non-null  float64
 6   부채        18345 non-null  float64
 7   당기순이익     18345 non-null  float64
 8   현금흐름      18345 non-null  float64
 9   매출액       18345 non-null  float64
 10  비유동자산     18345 non-null  float64
 11  매출총이익     18345 non-null  float64
 12  법인세비용     18345 non-null  float64
 13  이자비용      18345 non-null  float64
 14  감가상각비     18345 non-null  float64
 15  유동자산      18345 non-null  float64
 16  유동부채      18345 non-null  float64
 17  현금및현금성자산  18345 non-null  float64
 18  상장주식수     18345 non-null  float64
 19  시가총액      18345 non-null  float64
 20  pbr       18345 non-null  float

In [21]:
kospi = pd.read_csv("./데이터/kospi.csv", index_col=0)
kospi

,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
0,(주)DB하이텍,990,2013/12,1.0,6550.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1419.290470,-3499.345087,10239.594604,588.573379,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
1,(주)DB하이텍,990,2013/12,2.0,6890.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1492.963563,-3680.990481,10771.115545,619.125280,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
2,(주)DB하이텍,990,2013/12,3.0,6250.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1354.284799,-3339.069740,9770.605538,561.615820,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
3,(주)DB하이텍,990,2013/12,4.0,5970.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1293.612840,-3189.479415,9332.882410,536.455431,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
4,(주)DB하이텍,990,2013/12,5.0,8490.0,1.058392e+09,8.536351e+08,-83046768.0,28380938.0,4.937520e+08,...,3.340898e+11,1839.660471,-4535.792334,13272.390563,762.898930,-0.405587,-0.078465,-0.372473,0.084327,-0.000243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18340,효성티앤씨(주),298020,2022/12,8.0,313500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,895.220804,70724.639905,4624.426176,152.737768,0.012658,0.004441,0.008951,0.100036,0.000020
18341,효성티앤씨(주),298020,2022/12,9.0,259000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,739.592307,58429.606811,3820.498819,126.185270,0.012658,0.004441,0.008951,0.100036,0.000020
18342,효성티앤씨(주),298020,2022/12,10.0,261500.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,746.731229,58993.599155,3857.376221,127.403274,0.012658,0.004441,0.008951,0.100036,0.000020
18343,효성티앤씨(주),298020,2022/12,11.0,357000.0,4.319879e+09,2.804356e+09,19183248.0,293383061.0,8.882730e+09,...,1.514689e+12,1019.438044,80538.106686,5266.092966,173.931047,0.012658,0.004441,0.008951,0.100036,0.000020


In [20]:
((kospi["종가"] - kospi["종가"].shift(1)) / kospi["종가"].shift(1)) * 100

0              NaN
1         5.158730
2        -4.905660
3         0.000000
4        29.578755
           ...    
18340    -8.998549
18341   -17.384370
18342     0.965251
18343    36.520076
18344    -1.960784
Name: 종가, Length: 18345, dtype: float64

In [24]:
kospi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18345 entries, 0 to 18344
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회사명       18345 non-null  object 
 1   종목코드      18345 non-null  int64  
 2   회계년도      18345 non-null  object 
 3   거래월       18345 non-null  float64
 4   종가        18345 non-null  float64
 5   자산        18345 non-null  float64
 6   부채        18345 non-null  float64
 7   당기순이익     18345 non-null  float64
 8   현금흐름      18345 non-null  float64
 9   매출액       18345 non-null  float64
 10  비유동자산     18345 non-null  float64
 11  매출총이익     18345 non-null  float64
 12  법인세비용     18345 non-null  float64
 13  이자비용      18345 non-null  float64
 14  감가상각비     18345 non-null  float64
 15  유동자산      18345 non-null  float64
 16  유동부채      18345 non-null  float64
 17  현금및현금성자산  18345 non-null  float64
 18  상장주식수     18345 non-null  float64
 19  시가총액      18345 non-null  float64
 20  pbr       18345 non-null  float64

In [29]:
kospi.loc[kospi["회계년도"] == "2013/12"][kospi["거래월"] == 1.0].sort_values(by = ["pbr"], ascending=False)

C:\Users\duja1\AppData\Local\Temp\ipykernel_7600\472554982.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kospi.loc[kospi["회계년도"] == "2013/12"][kospi["거래월"] == 1.0].sort_values(by = ["pbr"], ascending=False)


,회사명,종목코드,회계년도,거래월,종가,자산,부채,당기순이익,현금흐름,매출액,...,시가총액,pbr,per,pcr,psr,roe,roa,roic,gpa,매직
17757,현대엘리베이터(주),17800,2013/12,1.0,98300.0,1.204332e+09,1.044260e+09,-342662157.0,1.043526e+08,1.066213e+09,...,6.516341e+11,8371.705395,-3910.779176,12841.809847,1256.856018,-2.140675,-0.284525,-0.525345,0.155419,-0.000597
15767,한국전력기술(주),52690,2013/12,1.0,80100.0,7.605037e+08,3.784537e+08,34407214.0,3.926585e+07,7.554839e+08,...,2.312310e+12,8013.146009,88976.166452,77966.528166,4052.266631,0.090059,0.045243,0.071854,0.197899,0.000020
10835,삼성엔지니어링(주),28050,2013/12,1.0,155500.0,5.928776e+09,5.023117e+09,-708658933.0,-1.230924e+09,9.806323e+09,...,2.640000e+12,6867.930797,-8777.141881,-5053.114895,634.284660,-0.782479,-0.119529,-0.166111,-0.092263,-0.000347
2664,(주)엘지생활건강,51900,2013/12,1.0,607000.0,3.435104e+09,1.959727e+09,365652541.0,3.895183e+08,4.326255e+09,...,8.558772e+12,6425.644203,25926.923831,24338.380022,2191.328330,0.247837,0.106446,0.372706,0.667937,0.000055
16715,한올바이오파마(주),9420,2013/12,1.0,8670.0,9.785160e+07,3.822538e+07,-21498897.0,-7.780099e+06,7.422831e+07,...,1.743216e+11,6078.506501,-16858.465272,-46585.320889,4882.751459,-0.360561,-0.219709,-0.350634,0.324475,-0.000118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14790,태광산업(주),3240,2013/12,1.0,968000.0,3.994993e+09,1.291162e+09,113939554.0,2.338045e+08,3.513353e+09,...,1.436286e+12,398.608962,9459.148840,4609.711834,306.764261,0.042140,0.028521,0.077424,0.242960,0.000102
8781,동국제강(주),1230,2013/12,1.0,13050.0,9.109805e+09,6.490860e+09,-118408748.0,9.999798e+07,6.690855e+09,...,8.129894e+11,308.065629,-6813.744745,8068.233065,120.583536,-0.045212,-0.012998,-0.031364,0.047462,-0.000141
1204,(주)두산,150,2013/12,1.0,128500.0,3.116604e+10,2.210806e+10,130167997.0,3.833616e+08,2.193653e+10,...,2.913049e+12,296.241006,20614.484246,6999.517976,122.323158,0.014371,0.004177,0.010575,0.118020,0.000073
5690,(주)한화,880,2013/12,1.0,33450.0,1.133226e+11,1.025467e+11,224668731.0,6.404167e+09,3.872497e+10,...,2.938382e+12,232.681876,11160.296649,391.521616,64.748140,0.020849,0.001983,0.002218,0.024414,0.000133


In [34]:
pbr_jan2013 = kospi.loc[kospi["회계년도"] == "2013/12"][kospi["거래월"] == 1.0].sort_values(by = ["pbr"], ascending=False)

C:\Users\duja1\AppData\Local\Temp\ipykernel_7600\3100724689.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pbr_jan2013 = kospi.loc[kospi["회계년도"] == "2013/12"][kospi["거래월"] == 1.0].sort_values(by = ["pbr"], ascending=False)
